This is a demo for a complete autoemulate pipeline for the Naghavi heart model 


you have a simulation , put your simulator in a simulator object 


In [21]:
from autoemulate.simulations.naghavi_cardiac_ModularCirc import extract_parameter_ranges
# Usage example:
parameters_range = extract_parameter_ranges('/Users/mfamili/work/ModularCirc/Tutorials/Tutorial_03/Parameters_01.json')
parameters_range

{'ao.r': (120.0, 360.0),
 'ao.c': (0.15, 0.44999999999999996),
 'art.r': (562.5, 1687.5),
 'art.c': (1.5, 4.5),
 'ven.r': (4.5, 13.5),
 'ven.c': (66.65, 199.95000000000002),
 'av.r': (3.0, 9.0),
 'mv.r': (2.05, 6.1499999999999995),
 'la.E_pas': (0.22, 0.66),
 'la.E_act': (0.225, 0.675),
 'la.v_ref': (5.0, 15.0),
 'la.k_pas': (0.01665, 0.07500000000000001),
 'lv.E_pas': (0.5, 1.5),
 'lv.E_act': (1.5, 4.5),
 'lv.v_ref': (5.0, 15.0),
 'lv.k_pas': (0.00999, 0.045)}

In [22]:
import pandas as pd
import numpy as np
from autoemulate.experimental_design import LatinHypercube


# Generate Latin Hypercube samples
N_samples = 100
lhd = LatinHypercube(list(parameters_range.values()))
sample_array = lhd.sample(N_samples)
sample_df = pd.DataFrame(sample_array, columns=parameters_range.keys())

print("Number of parameters:", sample_df.shape[1], "Number of samples from each parameter:", sample_df.shape[0])
sample_df.head()

Number of parameters: 16 Number of samples from each parameter: 100


,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,la.v_ref,la.k_pas,lv.E_pas,lv.E_act,lv.v_ref,lv.k_pas
0,320.000173,0.425209,1202.815446,2.124049,11.948106,128.700999,6.999587,5.975806,0.314472,0.601894,8.023972,0.066173,1.189717,2.284156,7.352354,0.038644
1,152.165702,0.166727,1687.115062,2.664814,5.676083,150.804108,7.109822,4.256010,0.659079,0.662586,6.199816,0.039927,1.280774,4.169770,7.913426,0.030310
2,155.309219,0.320244,714.880577,4.192800,8.184227,91.399466,5.965593,2.611346,0.238476,0.319026,8.455369,0.060301,1.092149,4.074149,13.592903,0.035100
3,356.314555,0.164968,729.055697,1.939564,10.154901,174.494224,6.245979,3.229471,0.648556,0.502750,6.451998,0.053916,1.004030,2.316866,5.567938,0.016481
4,269.616809,0.410619,1319.406820,3.911644,12.700824,189.504590,4.095905,2.409456,0.248593,0.447825,7.035255,0.022567,1.012218,4.436770,11.296382,0.021405


In [23]:
from autoemulate.simulations.naghavi_cardiac_ModularCirc import NaghaviSimulator
# Initialize simulator with specific outputs
simulator = NaghaviSimulator(
    parameters_range=parameters_range, 
    output_variables=['lv.P_i', 'lv.P_o'],  # Only the ones you're interested in
    n_cycles=300, 
    dt=0.001,
)

# Run batch simulations with the samples generated in Cell 1
results = simulator.run_batch_simulations(sample_df)

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)


Running simulations:   0%|          | 0/100 [00:00<?, ?sample/s]

Successfully completed 100/100 simulations (100.0%)


In [24]:
print("Output names:", simulator.output_names)
results_df

Output names: ['lv.P_i_min', 'lv.P_i_max', 'lv.P_i_mean', 'lv.P_i_range', 'lv.P_o_min', 'lv.P_o_max', 'lv.P_o_mean', 'lv.P_o_range']


,0,1,2,3,4,5,6,7
0,48.635415,52.943001,51.855790,4.307585,48.635415,52.943001,51.855790,4.307585
1,22.286430,32.508589,31.001445,10.222158,22.286430,32.508589,31.001445,10.222158
2,24.994511,27.416577,26.851495,2.422066,24.994511,27.416577,26.851495,2.422066
3,4.080873,12.833359,11.000329,8.752487,4.080873,12.833359,11.000329,8.752487
4,6.350436,23.013697,18.485702,16.663260,6.350436,23.013697,18.485702,16.663260
...,...,...,...,...,...,...,...,...
95,7.952982,20.647194,18.523564,12.694212,7.952982,20.647194,18.523564,12.694212
96,10.988945,24.907917,23.426794,13.918971,10.988945,24.907917,23.426794,13.918971
97,2.924778,10.173989,8.341672,7.249211,2.924778,10.173989,8.341672,7.249211
98,1.472123,5.897426,4.372783,4.425303,1.472123,5.897426,4.372783,4.425303


test your simulator with our test function to make sure it ios compatible wih AutoEmulate pipelien 

In [25]:
# need a test for the simulator 

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import autoemulate as ae
from tqdm import tqdm
import os

from autoemulate.experimental_design import LatinHypercube
from autoemulate.compare import AutoEmulate
from autoemulate.plotting import _predict_with_optional_std

preprocessing_methods = [{"name" : "VAE", "params" : {"reduced_dim": 2}},
                         {"name" : "PCA", "params" : {"reduced_dim": 2}}]
preprocessing_methods = [{"name" : "PCA", "params" : {"reduced_dim": 2}}]
em = AutoEmulate()
em.setup(sample_df, results, models=["gp"], scale_output = True, reduce_dim_output=True, preprocessing_methods=preprocessing_methods)


,Values
Simulation input shape (X),"(100, 16)"
Simulation output shape (y),"(100, 8)"
Proportion of data for testing (test_set_size),0.2
Scale input data (scale),True
Scaler (scaler),StandardScaler
Do hyperparameter search (param_search),False
Reduce input dimensionality (reduce_dim),False
Cross validator (cross_validator),KFold
Parallel jobs (n_jobs),1


In [27]:
best_model = em.compare()

Cross-validating:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
em.summarise_cv()

,preprocessing,model,short,fold,rmse,r2
0,PCA,GaussianProcess,gp,4,2.332748,0.891104
1,PCA,GaussianProcess,gp,1,2.121067,0.852880
2,PCA,GaussianProcess,gp,3,2.969978,0.849793
3,PCA,GaussianProcess,gp,2,3.101657,0.840500
4,PCA,GaussianProcess,gp,0,2.680941,0.831239


In [31]:
#em.plot_eval(model=best_model)
best_model

{'preprocessing': 'PCA',
 'model': 'GaussianProcess',
 'transformer': 'PCA(n_components=2)'}

In [ ]:
## 3) Evaluate the emulator (on the test set)
gp = em.get_model('GaussianProcess')
em.evaluate(gp)

,model,short,preprocessing,rmse,r2
0,GaussianProcess,gp,PCA,3.2901,0.8254


In [ ]:
gp
gp_final = em.refit(gp)
gp_final

InputOutputPipeline(regressor=Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model', GaussianProcess())]),
                    transformer=Pipeline(steps=[('scaler_output',
                                                 StandardScaler()),
                                                ('dim_reducer_output',
                                                 PCA(n_components=2))]))

In [ ]:
print("Available methods:", [method for method in dir(gp_final) if callable(getattr(gp_final, method))])

Available methods: ['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__str__', '__subclasshook__', '_build_request_for_signature', '_check_feature_names', '_check_n_features', '_ensure_2d', '_fit_transformer', '_get_default_requests', '_get_doc_link', '_get_metadata_request', '_get_param_names', '_get_tags', '_inverse_transform_with_std', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', '_validate_params', 'fit', 'get_metadata_routing', 'get_params', 'predict', 'score', 'set_params', 'set_score_request']


now run hjistory matching

In [ ]:
from autoemulate.history_matching import HistoryMatcher
# Define observed data with means and variances
observations = {
    'lv.P_i_min': (0.0, 0.1),   # Minimum of minimum LV pressure
    'lv.P_i_max': (20.0, 0.1),   # Maximum of minimum LV pressure
    'lv.P_i_mean': (15.0, 0.1),  # Mean of minimum LV pressure
    'lv.P_i_range': (15.0, 0.5), # Range of minimum LV pressure
    'lv.P_o_min': (1.0, 0.1),  # Minimum of maximum LV pressure
    'lv.P_o_max': (13.0, 0.1),  # Maximum of maximum LV pressure
    'lv.P_o_mean': (12.0, 0.1), # Mean of maximum LV pressure
    'lv.P_o_range': (20.0, 0.5)  # Range of maximum LV pressure
}
    
# Test generating samples
samples = simulator.generate_initial_samples(10)
print("Generated samples:", samples)

# Create history matcher
hm = HistoryMatcher(
    simulator=simulator,
    observations=observations,  # This needs both means and variances
    threshold=3.0
)

# Run history matching
all_samples, all_impl_scores, emulator = hm.run_history_matching(
    n_waves=20,
    n_samples_per_wave=10,
    use_emulator=True,
    initial_emulator=gp_final,
)

Generated samples: [{'ao.r': 120.0, 'ao.c': 0.31666666666666665, 'art.r': 1562.5, 'art.c': 1.8333333333333333, 'ven.r': 11.5, 'ven.c': 155.51666666666668, 'av.r': 9.0, 'mv.r': 2.5055555555555555, 'la.E_pas': 0.66, 'la.E_act': 0.4750000000000001, 'la.v_ref': 9.444444444444445, 'la.k_pas': 0.01665, 'lv.E_pas': 1.2777777777777777, 'lv.E_act': 3.833333333333333, 'lv.v_ref': 8.333333333333332, 'lv.k_pas': 0.00999}, {'ao.r': 360.0, 'ao.c': 0.24999999999999997, 'art.r': 562.5, 'art.c': 4.5, 'ven.r': 8.5, 'ven.c': 185.1388888888889, 'av.r': 3.0, 'mv.r': 2.05, 'la.E_pas': 0.3666666666666667, 'la.E_act': 0.675, 'la.v_ref': 5.0, 'la.k_pas': 0.042583333333333334, 'lv.E_pas': 0.6111111111111112, 'lv.E_act': 3.166666666666667, 'lv.v_ref': 11.666666666666666, 'lv.k_pas': 0.01388}, {'ao.r': 200.0, 'ao.c': 0.44999999999999996, 'art.r': 812.5, 'art.c': 4.166666666666666, 'ven.r': 10.5, 'ven.c': 170.32777777777778, 'av.r': 5.0, 'mv.r': 2.961111111111111, 'la.E_pas': 0.4644444444444445, 'la.E_act': 0.5750

In [ ]:
from autoemulate.history_matching_dashboard import HistoryMatchingDashboard
dashboard = HistoryMatchingDashboard(
    samples=all_samples_df,
    impl_scores=all_impl_scores,
    param_names=simulator.param_names,  # Use simulator.param_names instead of parameter_names
    output_names=simulator.output_names  # Use simulator.output_names instead of output_names
)
dashboard.display()